In [61]:
#load packages
%matplotlib inline
import pickle
from sklearn import preprocessing

#Load data
with open('msd_full.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

doscaling = 1
if (doscaling == 1):
    xscaler = preprocessing.StandardScaler().fit(msd_data['X_train'])
    #standardize feature values
    X_train = xscaler.transform(msd_data['X_train'])
    X_test = xscaler.transform(msd_data['X_test'])
else:
    X_train = msd_data['X_train']
    X_test = msd_data['X_test']

Y_train = msd_data['Y_train']
Y_test = msd_data['Y_test'].astype('float32')
X_test = X_test.astype('float32')

y_mean = Y_train.mean()
Y_train_keep = Y_train.copy()
Y_test_keep = Y_test.copy()
Y_train = Y_train - y_mean
Y_test = Y_test - y_mean


#validation is the last 10% of training, subtraining is the first 90% of training
nvalid = int(X_train.shape[0] * 0.1)
nsubtrain = X_train.shape[0] - nvalid

X_subtrain = X_train[0:nsubtrain, :].astype('float32')
X_valid = X_train[nsubtrain:, :].astype('float32')
Y_subtrain = Y_train[0:nsubtrain].astype('float32')
Y_valid = Y_train[nsubtrain:].astype('float32')

Y_subtrain_keep = Y_train_keep[0:nsubtrain].astype('float32')
Y_valid_keep = Y_train_keep[nsubtrain:].astype('float32')

print("X_train shape = ", X_train.shape)
print("X_subtrain shape = ", X_subtrain.shape)
print("X_valid shape = ", X_valid.shape)
print("Y_subtrain shape = ", Y_subtrain.shape)
print("Y_valid shape = ", Y_valid.shape)
print("X_test shape = ", X_test.shape)

X_train shape =  (463715, 90)
X_subtrain shape =  (417344, 90)
X_valid shape =  (46371, 90)
Y_subtrain shape =  (417344,)
Y_valid shape =  (46371,)
X_test shape =  (51630, 90)


In [2]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
model = sm.OLS(Y_train, sm.add_constant(X_train))
results = model.fit()

In [4]:
np.set_printoptions(suppress=True)
print(results.params[1:6])

[ 5.30975265 -2.88088114 -1.53234348  0.05737583 -0.33952889]


In [5]:
ypred = results.predict(sm.add_constant(X_test))
rms = np.sqrt(mean_squared_error(ypred, Y_test))
print("rms = ", rms)

rms =  9.510160684544399


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import time
import math

In [45]:
class MLP(nn.Module):
    def __init__(self, H):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(90, H),
            nn.ReLU(),
            nn.Linear(H, H),
            nn.ReLU(),
            nn.Linear(H, H),
            nn.ReLU(),
            nn.Linear(H, 1)
        )
    def forward(self, x):
        return self.mlp(x)

In [46]:
class SSE(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x, y):
        return torch.sum(torch.pow((x - y), 2))

In [47]:
class dataset(Dataset):
    def __init__(self, x, y=None):
        self.x = torch.FloatTensor(x)
        self.y = torch.FloatTensor(y)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [51]:
def training(H, device):
    model = MLP(H).to(device)
    loss = SSE()
    optimizer = torch.optim.SGD(model.parameters(), lr= 0.00001)
    num_epoch = 100

    flag = 0
    batch_cnt = 0
    best_step_cnt = 0
    best_rmse = math.inf

    for epoch in range(num_epoch):
        epoch_start_time = time.time()
        train_acc = 0.0
        train_loss = 0.0

        model.train()
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            train_pred = torch.reshape(model(data[0]), (len(data[0]), ))
            batch_loss = loss(train_pred, data[1])
            batch_loss.backward()
            optimizer.step()
            train_loss += batch_loss.item()

            batch_cnt += 1
            best_step_cnt += 1
            if batch_cnt % 100 == 0:
                train_pred = model(torch.FloatTensor(X_subtrain))
                train_rmse = np.sqrt(mean_squared_error(train_pred.detach().numpy(), Y_subtrain))
                valid_pred = model(torch.FloatTensor(X_valid))
                valid_rmse = np.sqrt(mean_squared_error(valid_pred.detach().numpy(), Y_valid))
                if valid_rmse < best_rmse:
                    print('Train: %f, Valid: %f' % (train_rmse, valid_rmse))
                    best_rmse = valid_rmse
                    best_step_cnt = 0
                    torch.save(model, 'best_H'+str(H)+'.pkl')
            if best_step_cnt >= 5000:
                flag = 1
                break
        if flag == 1:
            break

        print('[%03d/%03d] %2.2f sec(s) Loss: %3.6f' % \
          (epoch + 1, num_epoch, time.time()-epoch_start_time, train_loss/train_set.__len__()))

In [52]:
torch.manual_seed(0)
np.random.seed(0)

batch_size = 1000
train_set = dataset(X_subtrain, Y_subtrain)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [53]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    device = "cuda"
else:
    device = "cpu"

In [54]:
# H=45
training(45, device)

Train: 9.449331, Valid: 9.379689
Train: 9.105864, Valid: 9.029032
Train: 8.955000, Valid: 8.894309
Train: 8.880700, Valid: 8.834023
[001/100] 5.97 sec(s) Loss: 88.909528
Train: 8.864705, Valid: 8.833821
Train: 8.787385, Valid: 8.753699
Train: 8.715527, Valid: 8.694748
[002/100] 5.40 sec(s) Loss: 78.068211
Train: 8.701793, Valid: 8.686587
Train: 8.691593, Valid: 8.683795
Train: 8.682180, Valid: 8.679771
Train: 8.660342, Valid: 8.657649
[003/100] 5.68 sec(s) Loss: 76.482771
Train: 8.654021, Valid: 8.652126
Train: 8.611747, Valid: 8.625824
[004/100] 5.59 sec(s) Loss: 75.413951
Train: 8.592249, Valid: 8.619514
Train: 8.591910, Valid: 8.612529
[005/100] 5.50 sec(s) Loss: 74.702658
Train: 8.571875, Valid: 8.608444
[006/100] 6.04 sec(s) Loss: 74.236598
Train: 8.543302, Valid: 8.604204
Train: 8.526733, Valid: 8.602380
[007/100] 6.01 sec(s) Loss: 73.664182
Train: 8.512384, Valid: 8.587558
[008/100] 5.84 sec(s) Loss: 73.365816
[009/100] 5.57 sec(s) Loss: 73.037205
Train: 8.465723, Valid: 8.57221

In [56]:
model = torch.load('best_H45.pkl')
model.eval()
test_pred = model(torch.FloatTensor(X_test))
test_rmse = np.sqrt(mean_squared_error(test_pred.detach().numpy(), Y_test))
print(test_rmse)

8.8407135


In [57]:
# H=90
training(90, device)

Train: 9.187493, Valid: 9.140050
Train: 8.953436, Valid: 8.899212
Train: 8.864867, Valid: 8.812171
Train: 8.798080, Valid: 8.765556
[001/100] 8.30 sec(s) Loss: 86.777884
Train: 8.786918, Valid: 8.758759
Train: 8.733521, Valid: 8.703835
[002/100] 7.57 sec(s) Loss: 77.310654
Train: 8.644323, Valid: 8.633585
Train: 8.616182, Valid: 8.617191
[003/100] 7.59 sec(s) Loss: 75.560333
Train: 8.579114, Valid: 8.608191
Train: 8.539529, Valid: 8.586229
[004/100] 7.54 sec(s) Loss: 74.445112
Train: 8.507303, Valid: 8.558593
[005/100] 7.25 sec(s) Loss: 73.441095
Train: 8.459901, Valid: 8.555553
[006/100] 7.74 sec(s) Loss: 72.723072
Train: 8.426862, Valid: 8.552495
Train: 8.378786, Valid: 8.537673
[007/100] 7.30 sec(s) Loss: 72.153218
Train: 8.371078, Valid: 8.527810
[008/100] 7.18 sec(s) Loss: 71.471137
[009/100] 7.24 sec(s) Loss: 70.855340
[010/100] 7.18 sec(s) Loss: 70.377707
[011/100] 7.24 sec(s) Loss: 69.857937
Train: 8.247368, Valid: 8.522726
[012/100] 7.77 sec(s) Loss: 69.491703
[013/100] 7.19 s

In [58]:
model = torch.load('best_H90.pkl')
model.eval()
test_pred = model(torch.FloatTensor(X_test))
test_rmse = np.sqrt(mean_squared_error(test_pred.detach().numpy(), Y_test))
print(test_rmse)

8.825038


In [59]:
# H=180
training(180, device)

Train: 9.254824, Valid: 9.218730
Train: 8.959325, Valid: 8.910476
Train: 8.787598, Valid: 8.756402
[001/100] 13.13 sec(s) Loss: 86.119623
Train: 8.706698, Valid: 8.684003
Train: 8.660354, Valid: 8.654922
[002/100] 11.91 sec(s) Loss: 76.881483
Train: 8.618110, Valid: 8.629775
Train: 8.576658, Valid: 8.601912
Train: 8.545632, Valid: 8.589115
[003/100] 11.90 sec(s) Loss: 74.889701
Train: 8.515756, Valid: 8.584311
Train: 8.481317, Valid: 8.570221
[004/100] 12.04 sec(s) Loss: 73.592101
Train: 8.446346, Valid: 8.546415
Train: 8.398228, Valid: 8.531609
[005/100] 11.97 sec(s) Loss: 72.416991
Train: 8.342570, Valid: 8.515935
Train: 8.340707, Valid: 8.510673
Train: 8.308944, Valid: 8.509221
[006/100] 13.34 sec(s) Loss: 71.493427
Train: 8.286865, Valid: 8.504952
Train: 8.258041, Valid: 8.489493
[007/100] 11.97 sec(s) Loss: 70.606164
[008/100] 11.94 sec(s) Loss: 69.649325
[009/100] 11.95 sec(s) Loss: 68.691412
[010/100] 11.96 sec(s) Loss: 67.863106
[011/100] 12.11 sec(s) Loss: 66.913194
[012/100] 

In [60]:
model = torch.load('best_H180.pkl')
model.eval()
test_pred = model(torch.FloatTensor(X_test))
test_rmse = np.sqrt(mean_squared_error(test_pred.detach().numpy(), Y_test))
print(test_rmse)

8.86847


In [66]:
def training_q4(H, w, device):
    model = MLP(H).to(device)
    loss = SSE()
    optimizer = torch.optim.SGD(model.parameters(), lr= 0.00001, weight_decay=w)
    num_epoch = 100

    flag = 0
    batch_cnt = 0
    best_step_cnt = 0
    best_rmse = math.inf

    for epoch in range(num_epoch):
        epoch_start_time = time.time()
        train_acc = 0.0
        train_loss = 0.0

        model.train()
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            train_pred = torch.reshape(model(data[0]), (len(data[0]), ))
            batch_loss = loss(train_pred, data[1])
            batch_loss.backward()
            optimizer.step()
            train_loss += batch_loss.item()

            batch_cnt += 1
            best_step_cnt += 1
            if batch_cnt % 100 == 0:
                train_pred = model(torch.FloatTensor(X_subtrain))
                train_rmse = np.sqrt(mean_squared_error(train_pred.detach().numpy(), Y_subtrain))
                valid_pred = model(torch.FloatTensor(X_valid))
                valid_rmse = np.sqrt(mean_squared_error(valid_pred.detach().numpy(), Y_valid))
                if valid_rmse < best_rmse:
                    print('Train: %f, Valid: %f' % (train_rmse, valid_rmse))
                    best_rmse = valid_rmse
                    best_step_cnt = 0
                    torch.save(model, 'best_H'+str(H)+'_w'+str(w)+'.pkl')
            if best_step_cnt >= 5000:
                flag = 1
                break
        if flag == 1:
            break

        print('[%03d/%03d] %2.2f sec(s) Loss: %3.6f' % \
          (epoch + 1, num_epoch, time.time()-epoch_start_time, train_loss/train_set.__len__()))

In [67]:
H = [45, 90, 180]
w = [0.1, 0.2, 0.4]
for i in H:
    for j in w:
        training_q4(i, j, device)

Train: 9.292967, Valid: 9.232933
Train: 8.974434, Valid: 8.910072
Train: 8.927661, Valid: 8.873037
Train: 8.859359, Valid: 8.813032
[001/100] 5.36 sec(s) Loss: 88.143067
Train: 8.784674, Valid: 8.748822
Train: 8.749503, Valid: 8.726721
Train: 8.695510, Valid: 8.677515
[002/100] 5.33 sec(s) Loss: 77.840285
Train: 8.653134, Valid: 8.646994
[003/100] 5.39 sec(s) Loss: 76.156295
Train: 8.635962, Valid: 8.639280
Train: 8.628282, Valid: 8.630695
Train: 8.601198, Valid: 8.608505
[004/100] 5.36 sec(s) Loss: 75.232691
Train: 8.586085, Valid: 8.601299
[005/100] 5.35 sec(s) Loss: 74.630244
Train: 8.569460, Valid: 8.597558
Train: 8.530726, Valid: 8.587249
Train: 8.527550, Valid: 8.570234
[006/100] 5.48 sec(s) Loss: 74.094340
[007/100] 5.32 sec(s) Loss: 73.634709
[008/100] 5.28 sec(s) Loss: 73.270173
[009/100] 5.28 sec(s) Loss: 72.903444
[010/100] 5.35 sec(s) Loss: 72.639119
Train: 8.433726, Valid: 8.557466
[011/100] 5.28 sec(s) Loss: 72.456061
Train: 8.417287, Valid: 8.550654
[012/100] 5.54 sec(s)

In [68]:
H = [45, 90, 180]
w = [0.1, 0.2, 0.4]
for i in H:
    for j in w:
        model = torch.load('best_H'+str(i)+'_w'+str(j)+'.pkl')
        model.eval()
        test_pred = model(torch.FloatTensor(X_test))
        test_rmse = np.sqrt(mean_squared_error(test_pred.detach().numpy(), Y_test))
        print('H = '+str(i)+', weight_decay = '+str(j)+', test RMSE = '+str(test_rmse))

H = 45, weight_decay = 0.1, test RMSE = 8.839773
H = 45, weight_decay = 0.2, test RMSE = 8.833174
H = 45, weight_decay = 0.4, test RMSE = 8.821029
H = 90, weight_decay = 0.1, test RMSE = 8.804137
H = 90, weight_decay = 0.2, test RMSE = 8.792378
H = 90, weight_decay = 0.4, test RMSE = 8.856186
H = 180, weight_decay = 0.1, test RMSE = 8.815477
H = 180, weight_decay = 0.2, test RMSE = 8.814477
H = 180, weight_decay = 0.4, test RMSE = 8.924497


In [69]:
class MLP_q5(nn.Module):
    def __init__(self, H):
        super(MLP_q5, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(90, H),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(H, H),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(H, H),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(H, 1)
        )
    def forward(self, x):
        return self.mlp(x)

In [70]:
def training_q5(H, device):
    model = MLP_q5(H).to(device)
    loss = SSE()
    optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
    num_epoch = 100

    flag = 0
    batch_cnt = 0
    best_step_cnt = 0
    best_rmse = math.inf

    for epoch in range(num_epoch):
        epoch_start_time = time.time()
        train_acc = 0.0
        train_loss = 0.0

        model.train()
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            train_pred = torch.reshape(model(data[0]), (len(data[0]), ))
            batch_loss = loss(train_pred, data[1])
            batch_loss.backward()
            optimizer.step()
            train_loss += batch_loss.item()

            batch_cnt += 1
            best_step_cnt += 1
            if batch_cnt % 100 == 0:
                train_pred = model(torch.FloatTensor(X_subtrain))
                train_rmse = np.sqrt(mean_squared_error(train_pred.detach().numpy(), Y_subtrain))
                valid_pred = model(torch.FloatTensor(X_valid))
                valid_rmse = np.sqrt(mean_squared_error(valid_pred.detach().numpy(), Y_valid))
                if valid_rmse < best_rmse:
                    print('Train: %f, Valid: %f' % (train_rmse, valid_rmse))
                    best_rmse = valid_rmse
                    best_step_cnt = 0
                    torch.save(model, 'best_q5_H'+str(H)+'.pkl')
            if best_step_cnt >= 5000:
                flag = 1
                break
        if flag == 1:
            break

        print('[%03d/%03d] %2.2f sec(s) Loss: %3.6f' % \
          (epoch + 1, num_epoch, time.time()-epoch_start_time, train_loss/train_set.__len__()))

In [71]:
training_q5(90, device)

Train: 9.691115, Valid: 9.627952
Train: 9.403133, Valid: 9.360952
Train: 9.299331, Valid: 9.243391
Train: 9.222829, Valid: 9.173450
[001/100] 10.22 sec(s) Loss: 92.348402
Train: 9.176925, Valid: 9.119189
Train: 9.139065, Valid: 9.085758
Train: 9.106014, Valid: 9.038915
Train: 9.076671, Valid: 9.029928
[002/100] 9.82 sec(s) Loss: 83.602306
Train: 9.061790, Valid: 9.017046
Train: 9.046077, Valid: 8.989340
Train: 9.029800, Valid: 8.978065
Train: 9.017885, Valid: 8.971027
[003/100] 9.94 sec(s) Loss: 82.041320
Train: 9.002901, Valid: 8.954693
Train: 8.993168, Valid: 8.935554
[004/100] 9.85 sec(s) Loss: 81.112333
Train: 8.966873, Valid: 8.915466
[005/100] 9.69 sec(s) Loss: 80.640662
Train: 8.953935, Valid: 8.915342
Train: 8.939194, Valid: 8.909977
[006/100] 10.70 sec(s) Loss: 80.107932
Train: 8.928417, Valid: 8.882907
Train: 8.925254, Valid: 8.882612
[007/100] 9.71 sec(s) Loss: 79.802161
Train: 8.904022, Valid: 8.878821
Train: 8.900240, Valid: 8.875978
[008/100] 9.79 sec(s) Loss: 79.516106
T

In [73]:
model = torch.load('best_q5_H90.pkl')
model.eval()
test_pred = model(torch.FloatTensor(X_test))
test_rmse = np.sqrt(mean_squared_error(test_pred.detach().numpy(), Y_test))
print('H = '+str(i)+', test RMSE = '+str(test_rmse))

H = 180, test RMSE = 8.773399


In [74]:
H = [20, 45, 180, 360]
for i in H:
    training_q5(i, device)

Train: 10.611014, Valid: 10.576477
Train: 10.037332, Valid: 9.998349
Train: 9.801664, Valid: 9.760788
Train: 9.698590, Valid: 9.669986
[001/100] 5.25 sec(s) Loss: 103.827231
Train: 9.646609, Valid: 9.597734
Train: 9.602829, Valid: 9.538258
Train: 9.550961, Valid: 9.504905
[002/100] 5.11 sec(s) Loss: 92.321439
Train: 9.539949, Valid: 9.455110
Train: 9.494735, Valid: 9.435950
[003/100] 5.11 sec(s) Loss: 90.562939
Train: 9.485288, Valid: 9.427824
Train: 9.478751, Valid: 9.422654
Train: 9.465181, Valid: 9.414675
Train: 9.461243, Valid: 9.409054
[004/100] 5.12 sec(s) Loss: 89.946521
Train: 9.452930, Valid: 9.383854
Train: 9.449419, Valid: 9.380079
[005/100] 5.02 sec(s) Loss: 89.192891
Train: 9.424513, Valid: 9.369101
Train: 9.418166, Valid: 9.352162
[006/100] 5.23 sec(s) Loss: 88.937700
Train: 9.419656, Valid: 9.338206
Train: 9.412177, Valid: 9.337986
[007/100] 5.01 sec(s) Loss: 88.714940
Train: 9.397417, Valid: 9.334004
Train: 9.399323, Valid: 9.310922
[008/100] 5.12 sec(s) Loss: 88.528425

In [75]:
for i in H:
    model = torch.load('best_q5_H'+str(i)+'.pkl')
    model.eval()
    test_pred = model(torch.FloatTensor(X_test))
    test_rmse = np.sqrt(mean_squared_error(test_pred.detach().numpy(), Y_test))
    print('H = '+str(i)+', test RMSE = '+str(test_rmse))

H = 20, test RMSE = 9.038035
H = 45, test RMSE = 8.865167
H = 180, test RMSE = 8.751879
H = 360, test RMSE = 8.753275


In [108]:
class L2_L1(nn.Module):
    def __init__(self, z):
        super().__init__()
        self.z = z
        
    def forward(self, x, y):
        return self.z * torch.sum(torch.pow((x - y), 2)) + (1 - self.z) * torch.sum(torch.abs(x - y))
    

In [109]:
def training_q7(z, device):
    model = MLP_q5(90).to(device)
    loss = L2_L1(z)
    optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
    num_epoch = 100

    flag = 0
    batch_cnt = 0
    best_step_cnt = 0
    best_rmse = math.inf

    for epoch in range(num_epoch):
        epoch_start_time = time.time()
        train_acc = 0.0
        train_loss = 0.0

        model.train()
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            train_pred = torch.reshape(model(data[0]), (len(data[0]), ))
            batch_loss = loss(train_pred, data[1])
            batch_loss.backward()
            optimizer.step()
            train_loss += batch_loss.item()
            

            batch_cnt += 1
            best_step_cnt += 1
            if batch_cnt % 100 == 0:
                train_pred = model(torch.FloatTensor(X_subtrain))
                train_rmse = np.sqrt(mean_squared_error(train_pred.detach().numpy(), Y_subtrain))
                valid_pred = model(torch.FloatTensor(X_valid))
                valid_rmse = np.sqrt(mean_squared_error(valid_pred.detach().numpy(), Y_valid))
                if valid_rmse < best_rmse:
                    print('Train: %f, Valid: %f' % (train_rmse, valid_rmse))
                    best_rmse = valid_rmse
                    best_step_cnt = 0
                    torch.save(model, 'best_q7_z'+str(z)+'.pkl')
            if best_step_cnt >= 5000:
                flag = 1
                break
        if flag == 1:
            break

        print('[%03d/%03d] %2.2f sec(s) Loss: %3.6f' % \
          (epoch + 1, num_epoch, time.time()-epoch_start_time, train_loss/train_set.__len__()))

In [110]:
z = [0.0, 0.1, 0.5, 0.9, 1.0]
for i in z:
    training_q7(i, device)

Train: 10.696766, Valid: 10.671568
Train: 9.926058, Valid: 9.889107
Train: 9.692993, Valid: 9.645587
Train: 9.550718, Valid: 9.500651
[001/100] 9.76 sec(s) Loss: 6.764667
Train: 9.525843, Valid: 9.486466
Train: 9.432894, Valid: 9.376958
Train: 9.424075, Valid: 9.374808
Train: 9.373528, Valid: 9.350177
[002/100] 9.74 sec(s) Loss: 6.201927
Train: 9.327116, Valid: 9.292807
Train: 9.328636, Valid: 9.285329
[003/100] 9.70 sec(s) Loss: 6.099921
Train: 9.252521, Valid: 9.214709
Train: 9.263678, Valid: 9.203506
[004/100] 9.73 sec(s) Loss: 6.057398
[005/100] 9.66 sec(s) Loss: 6.027022
[006/100] 10.60 sec(s) Loss: 5.999294
[007/100] 9.63 sec(s) Loss: 5.986274
Train: 9.196467, Valid: 9.170694
[008/100] 9.87 sec(s) Loss: 5.970306
[009/100] 9.67 sec(s) Loss: 5.961331
[010/100] 9.71 sec(s) Loss: 5.950344
Train: 9.174382, Valid: 9.160086
[011/100] 9.78 sec(s) Loss: 5.939454
[012/100] 10.60 sec(s) Loss: 5.935695
[013/100] 9.68 sec(s) Loss: 5.925118
[014/100] 9.68 sec(s) Loss: 5.928199
[015/100] 9.77 s

In [111]:
for i in z:
    model = torch.load('best_q7_z'+str(i)+'.pkl')
    model.eval()
    test_pred = model(torch.FloatTensor(X_test))
    test_rmse = np.sqrt(mean_squared_error(test_pred.detach().numpy(), Y_test))
    print('H = '+str(i)+', test RMSE = '+str(test_rmse))

H = 0.0, test RMSE = 9.017365
H = 0.1, test RMSE = 8.78764
H = 0.5, test RMSE = 8.770102
H = 0.9, test RMSE = 8.802077
H = 1.0, test RMSE = 8.792296


In [112]:
class Customerized(nn.Module):
    def __init__(self, z):
        super().__init__()
        self.z = z
        
    def forward(self, x, y):
        return self.z * torch.sum(torch.pow((x - y), 2)) + (1 - self.z) * torch.sum(0.5 * torch.clamp(x - y, min=0.0) + 0.5 * torch.clamp(y - x, min=0.0))


In [113]:
def training_q8(z, device):
    model = MLP_q5(90).to(device)
    loss = Customerized(z)
    optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
    num_epoch = 100

    flag = 0
    batch_cnt = 0
    best_step_cnt = 0
    best_rmse = math.inf

    for epoch in range(num_epoch):
        epoch_start_time = time.time()
        train_acc = 0.0
        train_loss = 0.0

        model.train()
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            train_pred = torch.reshape(model(data[0]), (len(data[0]), ))
            batch_loss = loss(train_pred, data[1])
            batch_loss.backward()
            optimizer.step()
            train_loss += batch_loss.item()
            

            batch_cnt += 1
            best_step_cnt += 1
            if batch_cnt % 100 == 0:
                train_pred = model(torch.FloatTensor(X_subtrain))
                train_rmse = np.sqrt(mean_squared_error(train_pred.detach().numpy(), Y_subtrain))
                valid_pred = model(torch.FloatTensor(X_valid))
                valid_rmse = np.sqrt(mean_squared_error(valid_pred.detach().numpy(), Y_valid))
                if valid_rmse < best_rmse:
                    print('Train: %f, Valid: %f' % (train_rmse, valid_rmse))
                    best_rmse = valid_rmse
                    best_step_cnt = 0
                    torch.save(model, 'best_q8_z'+str(z)+'.pkl')
            if best_step_cnt >= 5000:
                flag = 1
                break
        if flag == 1:
            break

        print('[%03d/%03d] %2.2f sec(s) Loss: %3.6f' % \
          (epoch + 1, num_epoch, time.time()-epoch_start_time, train_loss/train_set.__len__()))

In [114]:
z = [0.1, 0.5, 0.9, 1.0]
for i in z:
    training_q8(i, device)

Train: 9.734010, Valid: 9.675782
Train: 9.448947, Valid: 9.391056
Train: 9.316045, Valid: 9.273231
Train: 9.244974, Valid: 9.204953
[001/100] 10.06 sec(s) Loss: 12.349944
Train: 9.186489, Valid: 9.135624
Train: 9.143822, Valid: 9.106217
Train: 9.123127, Valid: 9.072780
Train: 9.101375, Valid: 9.059466
[002/100] 9.86 sec(s) Loss: 11.249713
Train: 9.066452, Valid: 9.008919
Train: 9.038618, Valid: 9.004548
Train: 9.028793, Valid: 9.000247
[003/100] 9.86 sec(s) Loss: 11.030298
Train: 9.010324, Valid: 8.969580
Train: 8.998832, Valid: 8.946953
[004/100] 9.74 sec(s) Loss: 10.917136
Train: 8.978270, Valid: 8.940965
Train: 8.977426, Valid: 8.938670
Train: 8.975590, Valid: 8.933332
[005/100] 9.84 sec(s) Loss: 10.871514
Train: 8.949886, Valid: 8.920117
Train: 8.937027, Valid: 8.904657
[006/100] 10.76 sec(s) Loss: 10.805790
[007/100] 9.83 sec(s) Loss: 10.780014
Train: 8.912510, Valid: 8.882490
[008/100] 9.76 sec(s) Loss: 10.734988
Train: 8.896894, Valid: 8.862901
[009/100] 9.73 sec(s) Loss: 10.716

In [115]:
for i in z:
    model = torch.load('best_q8_z'+str(i)+'.pkl')
    model.eval()
    test_pred = model(torch.FloatTensor(X_test))
    test_rmse = np.sqrt(mean_squared_error(test_pred.detach().numpy(), Y_test))
    print('z = '+str(i)+', test RMSE = '+str(test_rmse))

z = 0.1, test RMSE = 8.760516
z = 0.5, test RMSE = 8.786251
z = 0.9, test RMSE = 8.77406
z = 1.0, test RMSE = 8.794384
